In [ ]:
import glob
import os
import shutil

from astropy import units as u
from astropy.io import fits
from astropy.nddata import Cutout2D
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bokeh.io import output_file, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Range1d
from bokeh.layouts import row, column, gridplot
from bokeh.models.widgets import Tabs, Panel

# Determine where the visualization will be rendered
output_notebook()

In [ ]:
path = glob.glob('/Volumes/wfc3u/final_postflash_2021/notebooks/*10_18*flc.fits')

for f in path: 
    print(f[72:])

In [ ]:
flc_files = glob.glob('/Volumes/wfc3u/final_postflash_2021/notebooks/i*.fits')

for f in flc_files: 
    print(f)

In [ ]:
updated_science_yearly = glob.glob('/Volumes/wfc3u/final_postflash_2021/notebooks/*10_18*flc.fits')

for f in updated_science_yearly: 
    print(f)
    image_data = fits.getdata(f,1)
    print('Min:', np.min(image_data))
    print('Max:', np.max(image_data))
    print('Mean:', np.mean(image_data))
    print('Stdev:', np.std(image_data))
    print('Median: ', np.median(image_data))


Create cutouts

In [ ]:
positions = [(220, 160), (900,700), (2100,1100), (3500,300), (3600,1700)]
size = (101, 101)
updated_science_yearly = glob.glob('/Volumes/wfc3u/final_postflash_2021/notebooks/*10_18*flc.fits')

min_pos = []
max_pos = []
mean_pos = []
median_pos = []
std_pos = []

for f in updated_science_yearly: 
    print(f)
    image_data = fits.getdata(f,4)
    means_yearly = []
    maxes_yearly = []
    mins_yearly = []
    medians_yearly = []
    stdevs_yearly = []
    for p in positions: 
        print(p)
        cutout = Cutout2D(image_data, p, size)
        cutout1 = cutout.data
        mins_yearly.append(np.min(cutout1))
        maxes_yearly.append(np.max(cutout1))
        means_yearly.append(np.mean(cutout1))
        medians_yearly.append(np.median(cutout1))
        stdevs_yearly.append(np.std(cutout1))
        print(np.std(cutout1))
    std_pos.append(stdevs_yearly)
    min_pos.append(mins_yearly)
    max_pos.append(maxes_yearly)
    mean_pos.append(means_yearly)
    median_pos.append(medians_yearly)

In [ ]:
filenames = []
years = []

science_yearly = glob.glob('/Volumes/wfc3u/final_postflash_2021/notebooks/*10_18*flc.fits')
for f in science_yearly:
    print(f[73:-9])
    filenames.append(f[73:-9])
    years.append(f[46:50])  

positions = [220, 900, 2100, 3500, 3600]
    
stdev_yearly = pd.DataFrame(std_pos, columns=positions, dtype = float, index=filenames)
stdev_yearly
years = pd.DataFrame(years, index=filenames)
years

stdev_yearly = pd.concat([stdev_yearly, years],1)
stdev_yearly

In [ ]:
x = list([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])
y = list(stdev_yearly[220].values)
y2 = list(stdev_yearly[900].values)
y3 = list(stdev_yearly[2100].values)
y4 = list(stdev_yearly[3500].values)
y5 = list(stdev_yearly[3600].values)
type(y)
p = figure(title = 'Checking Stdev')
p.line(x,y)
p.line(x,y2)
p.line(x,y3)
p.line(x,y4)
p.line(x,y5)
show(p)

In [ ]:
science_pipeline = glob.glob('/Volumes/wfc3u/final_postflash_2021/science-flc/*-pipe.fits')

positions = [(220, 160), (900,700), (2100,1100), (3500,300), (3600,1700)]
size = (101, 101) 

min_pos = []
max_pos = []
mean_pos = []
median_pos = []
std_pos = []

for f in science_pipeline: 
    print(f)
    image_data = fits.getdata(f,4)
    means_yearly = []
    maxes_yearly = []
    mins_yearly = []
    medians_yearly = []
    stdevs_yearly = []
    for p in positions: 
        cutout = Cutout2D(image_data, p, size)
        cutout1 = cutout.data
        mins_yearly.append(np.min(cutout1))
        maxes_yearly.append(np.max(cutout1))
        means_yearly.append(np.mean(cutout1))
        medians_yearly.append(np.median(cutout1))
        stdevs_yearly.append(np.std(cutout1))
    std_pos.append(stdevs_yearly)
    min_pos.append(mins_yearly)
    max_pos.append(maxes_yearly)
    mean_pos.append(means_yearly)
    median_pos.append(medians_yearly)
    
filenames = []

science_pipeline = glob.glob('/Volumes/wfc3u/final_postflash_2021/science-flc/*')
for f in science_pipeline:
    print(f[48:-14])
    filenames.append(f[48:-14]) 
    
positions_pipeline = ['pipe_220', 'pipe_900', 'pipe_2100', 'pipe_3500', 'pipe_3600']
    
stdev_pipeline = pd.DataFrame(std_pos, columns=positions_pipeline, dtype = float, index=filenames)
stdev_pipeline

In [ ]:
combine_by_file = pd.concat([stdev_yearly,stdev_pipeline],1)
combine_by_file

In [ ]:
combine_by_file['220_norm'] = (combine_by_file[220] - combine_by_file['pipe_220']) / combine_by_file['pipe_220']
combine_by_file['900_norm'] = (combine_by_file[900] - combine_by_file['pipe_900']) / combine_by_file['pipe_900']
combine_by_file['2100_norm'] = (combine_by_file[2100] - combine_by_file['pipe_2100']) / combine_by_file['pipe_2100']
combine_by_file['3500_norm'] = (combine_by_file[3500] - combine_by_file['pipe_3500']) / combine_by_file['pipe_3500']
combine_by_file['3600_norm'] = (combine_by_file[3600] - combine_by_file['pipe_3600']) / combine_by_file['pipe_3600']

combine_by_file

In [ ]:
sorted_by_year = combine_by_file.sort_values(by=[0])
sorted_by_year

In [ ]:
x = list([2012,2013,2015,2016,2018,2020])
#x2 = list([5,1,4,6,3,2])
x3 = list([1, 2, 3, 4, 5, 6])
y = list(sorted_by_year['220_norm'].values)
y2 = list(sorted_by_year['900_norm'].values)
y3 = list(sorted_by_year['2100_norm'].values)
y4 = list(sorted_by_year['3500_norm'].values)
y5 = list(sorted_by_year['3600_norm'].values)

y_c = [x for x in y if str(x) != 'nan']
y2_c = [x for x in y2 if str(x) != 'nan']
y3_c = [x for x in y3 if str(x) != 'nan']
y4_c = [x for x in y4 if str(x) != 'nan']
y5_c = [x for x in y5 if str(x) != 'nan']
p = figure(title = 'Checking Stdev')
p.line(x,y_c)
p.line(x,y2_c)
p.line(x,y3_c)
p.line(x,y4_c)
p.line(x,y5_c)
show(p)

In [ ]:
#plt.hist(cutout1.flatten(),alpha=0.5, range=(-20,20), bins=100)
plt.hist(cutout1_biyearly.flatten(),alpha=0.5, range=(-20,20), bins=100)
plt.hist(cutout1_curr.flatten(),alpha=0.5, range=(-20,20), bins=100)

In [ ]:
histogram = plt.hist(cutout1_shutter.flatten(), range=(-20,20), bins=100)

In [ ]:
postflash_data = pd.read_pickle('/Volumes/wfc3u/final_postflash_2021/notebooks/Feb_2022_all_postflash_flc.pkl')

In [ ]:
postflash_data.loc[postflash_data['filename']=='ielu05rmq_flc.fits']